# Loading libraries
Loading the libraries I will need and suppressing the warnings, as per advice from classes.

In [2]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
sns.set(color_codes=True) 
import sklearn.preprocessing as preproc
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from scipy import stats
import statistics as stats
from scipy.stats import poisson
from scipy.stats import binom
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
import math

import warnings
warnings.filterwarnings('ignore')

## Importing the dataset and combining two datasets
Two datasets (application_data and previous_application) are merged based on column that is common to both datasets - "SK_ID_CURR". I saved this new dataset as "merged_dataset".

In [145]:
df1 = pd.read_csv('application_data.csv')
df2 = pd.read_csv('previous_application.csv')

df = pd.merge(df1, df2, on='SK_ID_CURR', how='left')

df.to_csv('merged_dataset.csv', index=False)


In [146]:
df.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1430150,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,XNA,24.0,middle,Cash X-Sell: middle,365243.0,-618.0,72.0,-558.0,-528.0,1.0
1430151,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,Connectivity,0.0,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN
1430152,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,XNA,60.0,low_normal,Cash Street: low,NaN,NaN,NaN,NaN,NaN,NaN
1430153,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,XNA,36.0,low_normal,Cash X-Sell: low,365243.0,-756.0,294.0,-66.0,-64.0,0.0
1430154,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,Connectivity,6.0,high,POS mobile with interest,365243.0,-469.0,-319.0,-409.0,-400.0,0.0


In [147]:
df.info

<bound method DataFrame.info of          SK_ID_CURR  TARGET NAME_CONTRACT_TYPE_x CODE_GENDER FLAG_OWN_CAR  \
0            100002       1           Cash loans           M            N   
1            100003       0           Cash loans           F            N   
2            100003       0           Cash loans           F            N   
3            100003       0           Cash loans           F            N   
4            100004       0      Revolving loans           M            Y   
...             ...     ...                  ...         ...          ...   
1430150      456255       0           Cash loans           F            N   
1430151      456255       0           Cash loans           F            N   
1430152      456255       0           Cash loans           F            N   
1430153      456255       0           Cash loans           F            N   
1430154      456255       0           Cash loans           F            N   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_

## Removing NaN values.
First, due to high number of columns and rows, I have decided to remove columns that contain more than 20% of NaN values. First line of code was to calculate the percentage of NaN values in each column. Second line of the code was to get the list of columns with more than 20% NaN values and third line of the code was actual dropping those columns with more than 20% of NaN values. There were 64 columns that were falling into this condition and none of those columns was cruical for further work, so I could drop them.

In [148]:
nan_percentage = df.isna().mean() * 100

cols_with_high_nan = nan_percentage[nan_percentage > 20].index.tolist()

df = df.drop(cols_with_high_nan, axis=1)

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,XAP,New,Vehicles,POS,XNA,Stone,500.0,Auto technology,low_normal,POS other with interest
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1.0,XNA,low_normal,Cash X-Sell: low
2,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,XAP,Refreshed,Furniture,POS,XNA,Stone,1400.0,Furniture,middle,POS industry with interest
3,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,XAP,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200.0,Consumer electronics,middle,POS household with interest
4,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,XAP,New,Mobile,POS,XNA,Regional / Local,30.0,Connectivity,middle,POS mobile without interest


In [149]:
df.info

<bound method DataFrame.info of          SK_ID_CURR  TARGET NAME_CONTRACT_TYPE_x CODE_GENDER FLAG_OWN_CAR  \
0            100002       1           Cash loans           M            N   
1            100003       0           Cash loans           F            N   
2            100003       0           Cash loans           F            N   
3            100003       0           Cash loans           F            N   
4            100004       0      Revolving loans           M            Y   
...             ...     ...                  ...         ...          ...   
1430150      456255       0           Cash loans           F            N   
1430151      456255       0           Cash loans           F            N   
1430152      456255       0           Cash loans           F            N   
1430153      456255       0           Cash loans           F            N   
1430154      456255       0           Cash loans           F            N   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_

## Removing rows with NaN values
After I removed columns with more than 20% NaN values, now it was time to remove rows with NaN values. Since we had dataset with more than million rows, there was no issue with dropping certain amount of rows and reducing the size of the dataset.
First, I checked number of NaN values:

In [150]:
total_NaN_number = df.isna().sum().sum()
total_NaN_number

1623612

### After that, I dropped rows with NaN values:

In [151]:
df = df.dropna()
df.info

<bound method DataFrame.info of          SK_ID_CURR  TARGET NAME_CONTRACT_TYPE_x CODE_GENDER FLAG_OWN_CAR  \
0            100002       1           Cash loans           M            N   
4            100004       0      Revolving loans           M            Y   
20           100008       0           Cash loans           M            N   
21           100008       0           Cash loans           M            N   
22           100008       0           Cash loans           M            N   
...             ...     ...                  ...         ...          ...   
1430150      456255       0           Cash loans           F            N   
1430151      456255       0           Cash loans           F            N   
1430152      456255       0           Cash loans           F            N   
1430153      456255       0           Cash loans           F            N   
1430154      456255       0           Cash loans           F            N   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_

In [152]:
total_NaN_number = df.isna().sum().sum()

total_NaN_number

0

### Since we still have high number of columns (94), I decided to list all the columns and create new dataframe with columns I find relevant for further work.

In [153]:
column_names = df.columns.tolist()

print(column_names)

['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE_x', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT_x', 'AMT_ANNUITY_x', 'AMT_GOODS_PRICE_x', 'NAME_TYPE_SUITE_x', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START_x', 'HOUR_APPR_PROCESS_START_x', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DO

In [154]:
df = df[['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE_x', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT_x', 'AMT_ANNUITY_x', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'SK_ID_PREV', 'NAME_CONTRACT_TYPE_y', 'AMT_APPLICATION', 'AMT_CREDIT_y', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE']]

In [155]:
df.info

<bound method DataFrame.info of          SK_ID_CURR  TARGET NAME_CONTRACT_TYPE_x CODE_GENDER FLAG_OWN_CAR  \
0            100002       1           Cash loans           M            N   
4            100004       0      Revolving loans           M            Y   
20           100008       0           Cash loans           M            N   
21           100008       0           Cash loans           M            N   
22           100008       0           Cash loans           M            N   
...             ...     ...                  ...         ...          ...   
1430150      456255       0           Cash loans           F            N   
1430151      456255       0           Cash loans           F            N   
1430152      456255       0           Cash loans           F            N   
1430153      456255       0           Cash loans           F            N   
1430154      456255       0           Cash loans           F            N   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_

### Now we have new dataset with 29 columns and 1164707 rows.

## Removing duplicates
One more thing I decided to check on this dataset is number of duplicated rows, since I merged two datasets based on SK_ID_CURR column. I will check how many rows have duplicated ID and will remove those rows as well.

In [156]:
duplicates = df['SK_ID_CURR'].duplicated()
num_duplicates = duplicates.sum()
num_duplicates

932511

In [157]:
df = df.drop_duplicates(subset=['SK_ID_CURR'], keep='first')

df.info

<bound method DataFrame.info of          SK_ID_CURR  TARGET NAME_CONTRACT_TYPE_x CODE_GENDER FLAG_OWN_CAR  \
0            100002       1           Cash loans           M            N   
4            100004       0      Revolving loans           M            Y   
20           100008       0           Cash loans           M            N   
25           100009       0           Cash loans           F            Y   
32           100010       0           Cash loans           M            Y   
...             ...     ...                  ...         ...          ...   
1430130      456247       0           Cash loans           F            N   
1430139      456249       0           Cash loans           F            N   
1430143      456253       0           Cash loans           F            N   
1430145      456254       1           Cash loans           F            N   
1430147      456255       0           Cash loans           F            N   

        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_

### Our finalised dataset consist of 232196 rows in 29 columns.

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232196 entries, 0 to 1430147
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SK_ID_CURR                  232196 non-null  int64  
 1   TARGET                      232196 non-null  int64  
 2   NAME_CONTRACT_TYPE_x        232196 non-null  object 
 3   CODE_GENDER                 232196 non-null  object 
 4   FLAG_OWN_CAR                232196 non-null  object 
 5   FLAG_OWN_REALTY             232196 non-null  object 
 6   CNT_CHILDREN                232196 non-null  int64  
 7   AMT_INCOME_TOTAL            232196 non-null  float64
 8   AMT_CREDIT_x                232196 non-null  float64
 9   AMT_ANNUITY_x               232196 non-null  float64
 10  NAME_INCOME_TYPE            232196 non-null  object 
 11  NAME_EDUCATION_TYPE         232196 non-null  object 
 12  NAME_FAMILY_STATUS          232196 non-null  object 
 13  NAME_HOUSING_

## Replacing categorical variables with numerical values

In [159]:
df = df.replace({'N': 0, 'Y': 1})
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,SK_ID_PREV,NAME_CONTRACT_TYPE_y,AMT_APPLICATION,AMT_CREDIT_y,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE
0,100002,1,Cash loans,M,0,1,0,202500.0,406597.5,24700.5,...,1038818.0,Consumer loans,179055.0,179055.0,XAP,Approved,-606.0,XNA,XAP,New
4,100004,0,Revolving loans,M,1,1,0,67500.0,135000.0,6750.0,...,1564014.0,Consumer loans,24282.0,20106.0,XAP,Approved,-815.0,Cash through the bank,XAP,New
20,100008,0,Cash loans,M,0,1,0,99000.0,490495.5,27517.5,...,2218188.0,Consumer loans,162598.5,162598.5,XAP,Approved,-370.0,Cash through the bank,XAP,Repeater
25,100009,0,Cash loans,F,1,1,1,171000.0,1560726.0,41301.0,...,1413109.0,Consumer loans,98239.5,98239.5,XAP,Approved,-449.0,Cash through the bank,XAP,Repeater
32,100010,0,Cash loans,M,1,1,0,360000.0,1530000.0,42075.0,...,2349489.0,Consumer loans,247212.0,260811.0,XAP,Approved,-1070.0,Cash through the bank,XAP,New


### Getting number and names of the object columns

In [160]:
obj_cols = df.select_dtypes(include='object')

num_obj_cols = obj_cols.shape[1]

num_obj_cols

12

In [161]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
object_columns

['NAME_CONTRACT_TYPE_x',
 'CODE_GENDER',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'NAME_CONTRACT_TYPE_y',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_CLIENT_TYPE']

### Getting all unique values for each object column 

In [162]:
unique_values = df["NAME_CONTRACT_TYPE_x"].unique()
unique_values

array(['Cash loans', 'Revolving loans'], dtype=object)

In [163]:
unique_values = df["CODE_GENDER"].unique()
unique_values

array(['M', 'F', 'XNA'], dtype=object)

In [164]:
unique_values = df["NAME_INCOME_TYPE"].unique()
unique_values

array(['Working', 'State servant', 'Commercial associate', 'Pensioner',
       'Unemployed', 'Student'], dtype=object)

In [165]:
unique_values = df["NAME_EDUCATION_TYPE"].unique()
unique_values

array(['Secondary / secondary special', 'Higher education',
       'Incomplete higher', 'Lower secondary', 'Academic degree'],
      dtype=object)

In [166]:
unique_values = df["NAME_FAMILY_STATUS"].unique()
unique_values

array(['Single / not married', 'Married', 'Widow', 'Civil marriage',
       'Separated'], dtype=object)

In [167]:
unique_values = df["NAME_HOUSING_TYPE"].unique()
unique_values

array(['House / apartment', 'Rented apartment', 'Municipal apartment',
       'With parents', 'Office apartment', 'Co-op apartment'],
      dtype=object)

In [168]:
unique_values = df["NAME_CASH_LOAN_PURPOSE"].unique()
unique_values

array(['XAP', 'XNA', 'Repairs', 'Other', 'Building a house or an annex',
       'Urgent needs', 'Car repairs', 'Medicine', 'Furniture',
       'Education', 'Buying a used car',
       'Purchase of electronic equipment', 'Everyday expenses',
       'Wedding / gift / holiday', 'Buying a holiday home / land',
       'Buying a home', 'Payments on other loans', 'Business development',
       'Buying a new car', 'Journey', 'Gasification / water supply',
       'Buying a garage', 'Hobby', 'Money for a third person',
       'Refusal to name the goal'], dtype=object)

In [169]:
unique_values = df["NAME_CONTRACT_STATUS"].unique()
unique_values

array(['Approved', 'Refused', 'Canceled', 'Unused offer'], dtype=object)

In [170]:
unique_values = df["NAME_PAYMENT_TYPE"].unique()
unique_values

array(['XNA', 'Cash through the bank', 'Non-cash from your account',
       'Cashless from the account of the employer'], dtype=object)

In [171]:
unique_values = df["CODE_REJECT_REASON"].unique()
unique_values

array(['XAP', 'LIMIT', 'HC', 'SCO', 'VERIF', 'CLIENT', 'SCOFR', 'XNA',
       'SYSTEM'], dtype=object)

In [172]:
unique_values = df["NAME_CLIENT_TYPE"].unique()
unique_values

array(['New', 'Repeater', 'Refreshed', 'XNA'], dtype=object)

### Replacing all categorical variables with numeric values

In [174]:
df = df.replace({'Cash loans': 0, 'Revolving loans': 1, 'Consumer loans':2,
                'M': 0, 'F': 1, 'XNA': 3,
                'Unemployed': 0, 'Working': 1, 'State servant': 2, 'Commercial associate': 3, 'Pensioner': 4, 'Student': 5,
                'Secondary / secondary special': 0, 'Higher education': 1, 'Incomplete higher': 2, 'Lower secondary': 3, 'Academic degree': 4,
                'Single / not married': 0, 'Married': 1, 'Widow': 2, 'Civil marriage': 3, 'Separated': 4,
                'House / apartment': 0, 'Rented apartment': 1, 'Municipal apartment': 2, 'With parents': 3, 'Office apartment': 4, 'Co-op apartment': 5,
                'XAP': 0, 'Repairs': 1, 'Other': 2, 'Building a house or an annex': 4, 'Urgent needs': 5, 'Car repairs': 6, 'Medicine': 7, 'Furniture': 8, 'Education': 9, 'Buying a used car': 10, 'Purchase of electronic equipment': 11, 'Everyday expenses': 12, 'Wedding / gift / holiday': 13, 'Buying a holiday home / land': 14, 'Buying a home': 15, 'Payments on other loans': 16, 'Business development': 17, 'Buying a new car': 18, 'Journey': 19, 'Gasification / water supply': 20, 'Buying a garage': 21, 'Hobby': 22, 'Money for a third person': 23, 'Refusal to name the goal': 24,
                'Refused': 0, 'Approved': 1, 'Canceled': 2, 'Unused offer': 4,
                'Cash through the bank': 0, 'Non-cash from your account': 1, 'Cashless from the account of the employer': 2,
                'LIMIT': 1, 'HC': 2, 'SCO': 4, 'VERIF': 5, 'CLIENT': 6, 'SCOFR': 7, 'SYSTEM': 8,
                'New': 0, 'Repeater': 1, 'Refreshed': 2})
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,SK_ID_PREV,NAME_CONTRACT_TYPE_y,AMT_APPLICATION,AMT_CREDIT_y,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE
0,100002,1,0,0,0,1,0,202500.0,406597.5,24700.5,...,1038818.0,2,179055.0,179055.0,0,1,-606.0,3,0,0
4,100004,0,1,0,1,1,0,67500.0,135000.0,6750.0,...,1564014.0,2,24282.0,20106.0,0,1,-815.0,0,0,0
20,100008,0,0,0,0,1,0,99000.0,490495.5,27517.5,...,2218188.0,2,162598.5,162598.5,0,1,-370.0,0,0,1
25,100009,0,0,1,1,1,1,171000.0,1560726.0,41301.0,...,1413109.0,2,98239.5,98239.5,0,1,-449.0,0,0,1
32,100010,0,0,0,1,1,0,360000.0,1530000.0,42075.0,...,2349489.0,2,247212.0,260811.0,0,1,-1070.0,0,0,0


### Our dataset is now completely filled with numerical values only

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232196 entries, 0 to 1430147
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SK_ID_CURR                  232196 non-null  int64  
 1   TARGET                      232196 non-null  int64  
 2   NAME_CONTRACT_TYPE_x        232196 non-null  int64  
 3   CODE_GENDER                 232196 non-null  int64  
 4   FLAG_OWN_CAR                232196 non-null  int64  
 5   FLAG_OWN_REALTY             232196 non-null  int64  
 6   CNT_CHILDREN                232196 non-null  int64  
 7   AMT_INCOME_TOTAL            232196 non-null  float64
 8   AMT_CREDIT_x                232196 non-null  float64
 9   AMT_ANNUITY_x               232196 non-null  float64
 10  NAME_INCOME_TYPE            232196 non-null  int64  
 11  NAME_EDUCATION_TYPE         232196 non-null  int64  
 12  NAME_FAMILY_STATUS          232196 non-null  int64  
 13  NAME_HOUSING_

## List of replacements in dataset

#### NAME_CONTRACT_TYPE_x
Cash loans: 0, Revolving loans: 1, Consumer loans: 2
#### CODE_GENDER
M: 0, F: 1, XNA: 3
#### NAME_INCOME_TYPE
Unemployed: 0, Working: 1, State servant: 2, Commercial associate: 3, Pensioner: 4, Student: 5
#### NAME_EDUCATION_TYPE
Secondary / secondary special: 0, Higher education: 1, Incomplete higher: 2, Lower secondary: 3, Academic degree: 4
#### NAME_FAMILY_STATUS
Single / not married: 0, Married: 1, Widow: 2, Civil marriage: 3, Separated: 4
#### NAME_HOUSING_TYPE
House / apartment: 0, Rented apartment: 1, Municipal apartment: 2, With parents: 3, Office apartment: 4, Co-op apartment: 5
#### NAME_CASH_LOAN_PURPOSE
XAP: 0, 'Repairs': 1, 'Other': 2, XNA: 3, 'Building a house or an annex': 4, 'Urgent needs': 5, 'Car repairs': 6, 'Medicine': 7, 'Furniture': 8, 'Education': 9, 'Buying a used car': 10, 'Purchase of electronic equipment': 11, 'Everyday expenses': 12, 'Wedding / gift / holiday': 13, 'Buying a holiday home / land': 14, 'Buying a home': 15, 'Payments on other loans': 16, 'Business development': 17, 'Buying a new car': 18, 'Journey': 19, 'Gasification / water supply': 20, 'Buying a garage': 21, 'Hobby': 22, 'Money for a third person': 23, 'Refusal to name the goal': 24
#### NAME_CONTRACT_STATUS
'Refused': 0, 'Approved': 1, 'Canceled': 2, XNA: 3, 'Unused offer': 4
#### NAME_PAYMENT_TYPE
'Cash through the bank': 0, 'Non-cash from your account': 1, 'Cashless from the account of the employer': 2
#### CODE_REJECT_REASON
XAP: 0,'LIMIT': 1, 'HC': 2, XNA: 3, 'SCO': 4, 'VERIF': 5, 'CLIENT': 6, 'SCOFR': 7, 'SYSTEM': 8
#### NAME_CLIENT_TYPE
'New': 0, 'Repeater': 1, 'Refreshed': 2, XNA: 3

In [176]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,SK_ID_PREV,NAME_CONTRACT_TYPE_y,AMT_APPLICATION,AMT_CREDIT_y,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE
0,100002,1,0,0,0,1,0,202500.0,406597.5,24700.5,...,1038818.0,2,179055.0,179055.0,0,1,-606.0,3,0,0
4,100004,0,1,0,1,1,0,67500.0,135000.0,6750.0,...,1564014.0,2,24282.0,20106.0,0,1,-815.0,0,0,0
20,100008,0,0,0,0,1,0,99000.0,490495.5,27517.5,...,2218188.0,2,162598.5,162598.5,0,1,-370.0,0,0,1
25,100009,0,0,1,1,1,1,171000.0,1560726.0,41301.0,...,1413109.0,2,98239.5,98239.5,0,1,-449.0,0,0,1
32,100010,0,0,0,1,1,0,360000.0,1530000.0,42075.0,...,2349489.0,2,247212.0,260811.0,0,1,-1070.0,0,0,0


# 

# PCA
Before starting with PCA, I will scale the data.